================================================================================
                    **QUICK COPY-PASTE CODE TEMPLATES**
                     For Filling Your Notebook
================================================================================

Use these templates by copying and modifying for each country.
Based on YOUR existing code patterns from the notebook.

================================================================================
TEMPLATE 1: COMPLETE COUNTRY ANALYSIS (Tiers 1 & 2)
================================================================================


In [ ]:

# [COUNTRY NAME] ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS
print("="*80)
print("[COUNTRY] ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------
country_acled = acled[
    (acled['COUNTRY'] == '[CountryName]') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

country_acled['month'] = country_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
country_monthly = country_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(country_monthly)} months")
print(f"  Date range: {country_monthly['month'].min()} to {country_monthly['month'].max()}")
print(f"  Total events: {country_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {country_monthly['FATALITIES'].sum():,}")

# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
trends_files = {
    'search_term_1': 'TIER_FOLDER/google_trends_[country]_[term].csv',
    'search_term_2': 'TIER_FOLDER/google_trends_[country]_[term].csv',
    # Add all relevant files
}

trends_data = {}
for name, filepath in trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")

# 3. MERGE DATASETS
# -----------------
merged = country_monthly.copy()
for name, df in trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(trends_data)} search terms")

# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + corr_df.to_string(index=False))

# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

top_terms = corr_df.head(3)['Search Term'].tolist()

for term in top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())]
for idx, row in top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
merged_normalized = merged.copy()
merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
plot_data = []
for _, row in merged_normalized.iterrows():
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in top_terms:
        if term in row and pd.notna(row[term]):
            plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

plot_df = pd.DataFrame(plot_data)

# Main chart
chart = alt.Chart(plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': '[Country]: ACLED Events vs Google Search Interest (2020-2025)',
        'subtitle': 'Examining [brief description]',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('[country]_acled_vs_trends.html')
print(f"✓ Saved: [country]_acled_vs_trends.html")

# Display
chart

# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in top_terms:
    term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'[Country]: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"[country]_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ [Country] analysis complete!")

================================================================================
TEMPLATE 2: TIER 3 HASHTAG BATCH ANALYSIS
================================================================================

# TIER 3 HASHTAG ANALYSIS: [THEME]
print("="*80)
print("[THEME] HASHTAGS ANALYSIS")
print("="*80)

# Group related hashtags
hashtags = [
    "HashtagName1",
    "HashtagName2",
    "HashtagName3",
    # etc.
]

# Load all hashtags
hashtag_trends = {}
for hashtag in hashtags:
    # Try without .csv extension first
    try:
        df = pd.read_csv(f"TIER3_HASHTAGS/google_trends_{hashtag}", skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        hashtag_trends[hashtag] = df
        print(f"✓ Loaded: #{hashtag}")
    except FileNotFoundError:
        # Try with .csv extension
        try:
            df = pd.read_csv(f"TIER3_HASHTAGS/google_trends_{hashtag}.csv", skiprows=1)
            df.columns = ['month', 'value']
            df['month'] = pd.to_datetime(df['month'])
            df['value'] = df['value'].replace('<1', '0.5')
            df['value'] = pd.to_numeric(df['value'], errors='coerce')
            hashtag_trends[hashtag] = df
            print(f"✓ Loaded: #{hashtag}")
        except:
            print(f"✗ Skipping #{hashtag} - file not found (may be removed)")

# Create combined visualization
if len(hashtag_trends) > 0:
    # Reshape data
    plot_data = []
    for hashtag, df in hashtag_trends.items():
        for _, row in df.iterrows():
            plot_data.append({
                'month': row['month'],
                'hashtag': f'#{hashtag}',
                'value': row['value']
            })
    
    plot_df = pd.DataFrame(plot_data)
    
    # Multi-line chart
    chart = alt.Chart(plot_df).mark_line(strokeWidth=2, point=True).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
        y=alt.Y('value:Q', title='Search Interest (0-100)', scale=alt.Scale(domain=[0, 100])),
        color=alt.Color('hashtag:N', title='Hashtag'),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('hashtag:N', title='Hashtag'),
            alt.Tooltip('value:Q', title='Search Interest', format='.0f')
        ]
    ).properties(
        width=1400,
        height=450,
        title=f'[Theme] Hashtags: Search Interest Over Time (2020-2025)'
    ).interactive()
    
    chart.save('[theme]_hashtags.html')
    print(f"\n✓ Saved: [theme]_hashtags.html")
    
    # Display
    chart
else:
    print("\n⚠ No hashtags loaded for this group")

print("\n✓ Hashtag analysis complete!")

================================================================================
SPECIFIC COUNTRY CODE BLOCKS (Copy-Paste Ready)
================================================================================

# BRAZIL
# ------
trends_files = {
    'Brazil violence': 'TIER1_COUNTRIES/google_trends_Brazil_violence.csv',
    'Brazil gangs': 'TIER1_COUNTRIES/google_trends_Brazil_gangs.csv',
    'Brazil conflict': 'TIER1_COUNTRIES/google_trends_Brazil_conflict.csv'
}
# Major events: October 2022 election, January 2023 riots

# PAKISTAN
# --------
trends_files = {
    'Pakistan protests': 'TIER1_COUNTRIES/google_trends_Pakistan_protests.csv',
    'Pakistan violence': 'TIER1_COUNTRIES/google_trends_Pakistan_violence.csv',
    'Pakistan demonstrations': 'TIER1_COUNTRIES/google_trends_Pakistan_demonstrations.csv',
    'Pakistan unrest': 'TIER1_COUNTRIES/google_trends_Pakistan_unrest.csv'
}
# Major events: May 2023 Imran Khan arrest

# SYRIA
# -----
trends_files = {
    'Syria conflict': 'TIER1_COUNTRIES/google_trends_Syria_conflict.csv',
    'Syria war': 'TIER1_COUNTRIES/google_trends_Syria_war.csv',
    'Syria military': 'TIER1_COUNTRIES/google_trends_Syria_military.csv',
    'Syria bombing': 'TIER1_COUNTRIES/google_trends_Syria_bombing.csv'
}
# Major events: February 2023 earthquake

# YEMEN
# -----
trends_files = {
    'Yemen conflict': 'TIER1_COUNTRIES/google_trends_Yemen_conflict.csv',
    'Yemen war': 'TIER1_COUNTRIES/google_trends_Yemen_war.csv',
    'Yemen violence': 'TIER1_COUNTRIES/google_trends_Yemen_violence.csv',
    'Houthi Yemen': 'TIER1_COUNTRIES/google_trends_Houthi_Yemen.csv'
}
# Major events: January 2024 Red Sea attacks

# PALESTINE/GAZA
# --------------
trends_files = {
    'Palestine conflict': 'TIER1_COUNTRIES/google_trends_Palestine_conflict.csv',
    'Israel Palestine': 'TIER1_COUNTRIES/google_trends_Israel_Palestine.csv',
    'Gaza conflict': 'TIER1_COUNTRIES/google_trends_Gaza_conflict.csv',
    'Gaza ceasefire': 'TIER1_COUNTRIES/google_trends_Gaza_ceasefire.csv'
}
# Major events: October 7, 2023 attacks

# IRAQ
# ----
trends_files = {
    'Iraq conflict': 'TIER2_COUNTRIES/google_trends_Iraq_conflict.csv',
    'Iraq war': 'TIER2_COUNTRIES/google_trends_Iraq_war.csv',
    'Iraq military': 'TIER2_COUNTRIES/google_trends_Iraq_military.csv'
}

# FRANCE
# ------
trends_files = {
    'France protests': 'TIER2_COUNTRIES/google_trends_France_protests.csv',
    'French demonstrations': 'TIER2_COUNTRIES/google_trends_French_demonstrations.csv'
}

# RUSSIA
# ------
trends_files = {
    'Russia military': 'TIER2_COUNTRIES/google_trends_Russia_military.csv',
    'Russia conflict': 'TIER2_COUNTRIES/google_trends_Russia_conflict.csv'
}

# SOUTH KOREA (note: space in filename!)
# -----------
trends_files = {
    'South Korea protests': 'TIER2_COUNTRIES/google_trends_South_Korea protests.csv',
    'Korea demonstrations': 'TIER2_COUNTRIES/google_trends_Korea_demonstrations.csv'
}

# TURKEY
# ------
trends_files = {
    'Turkey conflict': 'TIER2_COUNTRIES/google_trends_Turkey_conflict.csv',
    'Turkey violence': 'TIER2_COUNTRIES/google_trends_Turkey_violence.csv',
    'Turkey military': 'TIER2_COUNTRIES/google_trends_Turkey_military.csv'
}

# COLOMBIA
# --------
trends_files = {
    'Colombia violence': 'TIER2_COUNTRIES/google_trends_Colombia_violence.csv',
    'Colombia conflict': 'TIER2_COUNTRIES/google_trends_Colombia_conflict.csv'
}

# LEBANON
# -------
trends_files = {
    'Lebanon conflict': 'TIER2_COUNTRIES/google_trends_Lebanon_conflict.csv',
    'Lebanon protests': 'TIER2_COUNTRIES/google_trends_Lebanon_protests.csv',
    'Lebanon crisis': 'TIER2_COUNTRIES/google_trends_Lebanon_crisis.csv'
}

# NIGERIA
# -------
trends_files = {
    'Nigeria insurgency': 'TIER2_COUNTRIES/google_trends_Nigeria_insurgency.csv',
    'Nigeria violence': 'TIER2_COUNTRIES/google_trends_Nigeria_violence.csv',
    'Boko Haram': 'TIER2_COUNTRIES/google_trends_Boko_Haram.csv'
}

# ITALY
# -----
trends_files = {
    'Italy protests': 'TIER2_COUNTRIES/google_trends_Italy_protests.csv',
    'Italian demonstrations': 'TIER2_COUNTRIES/google_trends_Italian_demonstrations.csv'
}

# AFGHANISTAN
# -----------
trends_files = {
    'Afghanistan conflict': 'TIER2_COUNTRIES/google_trends_Afghanistan_conflict.csv',
    'Afghanistan military': 'TIER2_COUNTRIES/google_trends_Afghanistan_military.csv'
}
# Note: Taliban.csv is separate, load individually

================================================================================
END OF TEMPLATES
================================================================================

TIP: Copy Template 1 for each country, modify the country name and file paths,
then run. Repeat for all countries in Tiers 1 and 2. Use Template 2 for
grouping Tier 3 hashtags by theme.

